# 1. Variational Auto Encoder

## 1.1. Latent Variable Models

* **Discrete LV**

>$$p(\mathbf{x}) = \sum^M_{m=1} P(c_m) p(\mathbf{x}|c_m)$$

* **Continuous LV**

>$$p(\mathbf{x}) = \int p(\mathbf{x}|\mathbf{z})p(\mathbf{z})d\mathbf{z}$$

>* $p(\mathbf{z}) = \mathcal{N}(\mathbf{z};\mathbf{0},\mathbf{I})$
>* $p(\mathbf{x}|\mathbf{z}) = \mathcal{N}(\mathbf{x};\mathbf{f}(\mathbf{z}),\sigma^2 \mathbf{I})$

* **Factor Analysis**, $\mathbf{f}(\mathbf{z}) = \mathbf{Az}$

>* **Auxiliary function:**

>$$\mathcal{Q}(\boldsymbol{\lambda}, \tilde{\boldsymbol{\lambda}}) = \int p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda}) \log (p(\mathbf{x}|\mathbf{z};\boldsymbol{\lambda})) d\mathbf{z}$$

>* **General mapping using NN**

>$$p(\mathbf{x}|\mathbf{z};\boldsymbol{\lambda}) = \mathcal{N}(\mathbf{x};\mathbf{f}(\mathbf{z};\boldsymbol{\lambda}),\sigma^2 \mathbf{I})$$

>* No simple closed-form solution $\rightarrow$ adopt **variational** approaches

## 1.2. KL Divergence

* **Definition**

>$$\mathcal{KL}(p(\mathbf{x})||q(\mathbf{x})) = \int p(\mathbf{x}) \log \left( \frac{p(\mathbf{x})}{q(\mathbf{x})} \right) d\mathbf{x} = - \int p(\mathbf{x}) \log \left( \frac{q(\mathbf{x})}{p(\mathbf{x})} \right) d\mathbf{x}$$

* **Inequality** (use $\log y \leq y-1$)

>$$\int p(\mathbf{x}) \left( \frac{q(\mathbf{x})}{p(\mathbf{x})} \right) d\mathbf{x} \leq
\int p(\mathbf{x}) \left( \frac{q(\mathbf{x})}{p(\mathbf{x})} - 1 \right) d\mathbf{x} = 0$$

* **KL Divergence for Gaussians**

>$$\mathcal{KL}(p(\mathbf{x})||q(\mathbf{x})) = \frac{1}{2} \left( \text{tr} (\boldsymbol{\Sigma}^{-1}_2 \boldsymbol{\Sigma}_1 - \mathbf{I}) + (\mu_1 - \mu_2)^T \boldsymbol{\Sigma}^{-1}_2 (\mu_1 - \mu_2) + \log \left( \frac{|\boldsymbol{\Sigma}_2|}{|\boldsymbol{\Sigma}_1|} \right) \right)$$

## 1.3. Variational EM

* **EM: Auxiliary Function Maximization**

>$$\boldsymbol{\lambda}^{(k+1)} = \text{argmax}_{\boldsymbol{\lambda}} \left( \mathcal{Q}(\boldsymbol{\lambda}^{(k)},\boldsymbol{\lambda}) \right) = \text{argmax}_{\boldsymbol{\lambda}} \left( \int p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda}^{(k)}) \log (p(\mathbf{x},\mathbf{z};\boldsymbol{\lambda})) d\mathbf{z} \right)$$

>$=$ **expected value of the log joint distribution**

>\begin{align}
\log (p(\mathbf{x},\mathbf{z};\boldsymbol{\lambda})) &= \log (p(\mathbf{x}|\mathbf{z};\boldsymbol{\lambda})) + \log (p(\mathbf{z})) \\
&= \log (\mathcal{N} (\mathbf{x}; \mathbf{Az}, \boldsymbol{\Sigma}_{\text{diag}})) + \log (\mathcal{N}(\mathbf{z};\mathbf{0},\mathbf{I})) \\
&= -\frac{1}{2} (\mathbf{z}^T \mathbf{A}^T \boldsymbol{\Sigma}^{-1}_{\text{diag}} \mathbf{Az} - 2\mathbf{z}^T \mathbf{A}^T \boldsymbol{\Sigma}^{-1}_{\text{diag}} \mathbf{x} + \mathbf{x}^T \boldsymbol{\Sigma}^{-1}_{\text{diag}} \mathbf{x}) - \frac{1}{2} \log (|\boldsymbol{\Sigma}_{\text{diag}}|) + C
\end{align}


* **Estimate $\boldsymbol{\lambda}$** by maximizing the **Log-likelihood** $\mathcal{L}(\boldsymbol{\lambda})$

>\begin{align}
\mathcal{L}(\boldsymbol{\lambda}) &= \log (p(\mathbf{x};\boldsymbol{\lambda})) \\
&= \int p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda}) \log (p(\mathbf{x};\boldsymbol{\lambda})) d\mathbf{z} \\
&= \int p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda}) \log \left( \frac{p(\mathbf{x};\boldsymbol{\lambda})p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda})}{p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda})} \right) d\mathbf{z} \\
&= \bigg\langle \log \left( \frac{p(\mathbf{x},\mathbf{z};\boldsymbol{\lambda})}{p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda})} \right) \bigg\rangle_{p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda})}
\end{align}

>* **Need to know** $p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda})$ $\rightarrow$ use any valid distribution $q(\mathbf{z};\tilde{\boldsymbol{\lambda}})$

>\begin{align}
\mathcal{L}(\boldsymbol{\lambda}) &= \bigg\langle \log \left( \frac{p(\mathbf{x},\mathbf{z};\boldsymbol{\lambda})}{p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda})} \right) \bigg\rangle_{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})} \\
&= \bigg\langle \log \left( \frac{p(\mathbf{x},\mathbf{z};\boldsymbol{\lambda})}{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})} \right) \bigg\rangle_{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})}
+ \bigg\langle \log \left( \frac{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})}{p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda})} \right) \bigg\rangle_{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})} \\
&\geq \bigg\langle \log \left( \frac{p(\mathbf{x},\mathbf{z};\boldsymbol{\lambda})}{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})} \right) \bigg\rangle_{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})}
=\mathcal{F} (q(\mathbf{z};\boldsymbol{\lambda}),\boldsymbol{\lambda})
\end{align}

* **EM Revisited**

>* **Set** $q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k)}) = p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda}^{(k)})$

>$$\mathcal{L}(\boldsymbol{\lambda}^{(k)}) = \mathcal{F} (q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k)}),\boldsymbol{\lambda}^{(k)})$$

>* **Maximize $\mathcal{F} (q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k)}),\boldsymbol{\lambda})$ to find $\boldsymbol{\lambda}^{(k+1)}$**

>$$\boldsymbol{\lambda}^{(k+1)} = \text{argmax}_{\boldsymbol{\lambda}} \Bigg\{ \bigg\langle \log \left( \frac{p(\mathbf{x},\mathbf{z};\boldsymbol{\lambda})}{q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k)})} \right) \bigg\rangle_{q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k)})} \Bigg\}$$

>* **Minimize KL divergence for variational approximation**

>$$q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k+1)}) = \text{argmin}_{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})} 
\Bigg\{ \bigg\langle
\log \left(
\frac{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})}{p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda}^{(k+1)})}
\right)
\bigg\rangle_{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})}\Bigg\}$$

>* **Which occurs at $q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k+1)}) = p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda}^{(k+1)})$**

* **EM Guarantees:**

>$$\mathcal{L}(\boldsymbol{\lambda}^{(k)}) = \mathcal{F} \left( q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k)}), \boldsymbol{\lambda}^{(k)} \right)
\leq \mathcal{F} \left( q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k)}), \boldsymbol{\lambda}^{(k+1)} \right)
\leq \mathcal{L}(\boldsymbol{\lambda}^{(k+1)})$$

>* provided $q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k)}) = p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda}^{(k)})$


* **Variational EM** (no guarantees)

>$$\mathcal{L}(\boldsymbol{\lambda}^{(k)}) \geq \mathcal{F} \left( q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k)}), \boldsymbol{\lambda}^{(k)} \right)
\leq \mathcal{F} \left( q(\mathbf{z};\tilde{\boldsymbol{\lambda}}^{(k)}), \boldsymbol{\lambda}^{(k+1)} \right)
\leq \mathcal{L}(\boldsymbol{\lambda}^{(k+1)})$$

>* Allows any form of $q(\mathbf{z};\tilde{\boldsymbol{\lambda}})$ / e.g. mean-field approximation, $q(\mathbf{z};\tilde{\boldsymbol{\lambda}}) = \prod^n_{i=1} q_i (z_i ; \tilde{\boldsymbol{\lambda}})$

## 1.4. Variational Auto Encoder

* **Variational Auto Encoder**

>$$p(\mathbf{x};\boldsymbol{\lambda}) = \int p(\mathbf{x}|\mathbf{z};\boldsymbol{\lambda})p(\mathbf{z})d\mathbf{z}
= \int \mathcal{N} (\mathbf{x};\mathbf{f}(\mathbf{z};\boldsymbol{\lambda}), \sigma^2 \mathbf{I}) p(\mathbf{z}) 
d\mathbf{z}$$

>* $\mathbf{f}(\mathbf{z};\boldsymbol{\lambda})$: cannot compute integral
>* VEM (normally) not possible for DNN

* **(Stochastic) Gradient Descent** - approximate gradient by lower-bound gradient

>\begin{align}
\nabla \mathcal{L}(\boldsymbol{\lambda}) &\approx \nabla \left( \bigg\langle
\log \left( \frac{p(\mathbf{x},\mathbf{z};\boldsymbol{\lambda})}{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})}
\right)
\bigg \rangle_{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})} \right) \\
&= \nabla \left(
\langle \log (p(\mathbf{x}|\mathbf{z};\boldsymbol{\lambda})) \rangle_{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})}
+ \bigg\langle \log \left( \frac{p(\mathbf{z})}{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})} \right)
\bigg\rangle_{q(\mathbf{z};\tilde{\boldsymbol{\lambda}})}
\right)
\end{align}

>* $p(\mathbf{x}|\mathbf{z};\boldsymbol{\lambda}) = \mathcal{N}(\mathbf{x};\mathbf{f}(\mathbf{z};\boldsymbol{\lambda}),\sigma^2 \mathbf{I})$
>* $p(\mathbf{z})$: also Gaussian
>* **Iterative Optimization**
>  * Optimize $\boldsymbol{\lambda}$ (model parameters)
>  * Optimize $\tilde{\boldsymbol{\lambda}}$ (variational approximation)

* **Variational Form**

>* Introduce dependence on $\mathbf{x}$

>$$q(\mathbf{z};\tilde{\boldsymbol{\lambda}}) \rightarrow q(\mathbf{z}|\mathbf{x};\tilde{\boldsymbol{\lambda}}) = \mathcal{N}(\mathbf{z};\mathbf{f}_\boldsymbol{\mu} (\mathbf{x};\tilde{\boldsymbol{\lambda}}), \mathbf{f}_\boldsymbol{\Sigma} (\mathbf{x};\tilde{\boldsymbol{\lambda}}))$$

>* Everything is Gaussian $\rightarrow$ rewrite $\mathcal{L}(\boldsymbol{\lambda})$

>$$\mathcal{L}(\boldsymbol{\lambda}) = 
\Bigg\langle \log \left( 
\frac{q(\mathbf{z}|\mathbf{x};\tilde{\boldsymbol{\lambda}})}
{p(\mathbf{z}|\mathbf{x};\boldsymbol{\lambda})} \right)
+ \log (p(\mathbf{x}|\mathbf{z};\boldsymbol{\lambda})) 
+ \log \left( \frac{p(\mathbf{z})}
{q(\mathbf{z}|\mathbf{x};\tilde{\boldsymbol{\lambda}})} 
\right) \Bigg\rangle_{q(\mathbf{z}|\mathbf{x};\tilde{\boldsymbol{\lambda}})}$$

>* **1st Term: error**
>  * Often neglected to yield the gradient lower-bound
>* **2nd Term: decoding** (compute probability given $\mathbf{z}$)
>  * Difficult if $\mathbf{f}(\mathbf{z})$ is non-linear
>* **3rd Term: encoding** (encode information about $\mathbf{x}$ into $\mathbf{z}$)
>  * (-) **KL Divergence between Gaussians** $\rightarrow$ closed-form solution

* **Monte-Carlo Approximation** and **Reparameterization Trick**

>\begin{align}
\langle \log (p(\mathbf{x}|\mathbf{z};\boldsymbol{\lambda})) \rangle_{q(\mathbf{z}|\mathbf{x};\tilde{\boldsymbol{\lambda}})}
&\approx \frac{1}{K} \sum^K_{i=1} \log (p(\mathbf{x}|\mathbf{z}^{(i)};\boldsymbol{\lambda})) \\
q(\mathbf{z}|\mathbf{x};\tilde{\boldsymbol{\lambda}}) &= 
\mathcal{N}(\mathbf{z};\mathbf{f}_\mathbf{\mu} (\mathbf{x};\tilde{\boldsymbol{\lambda}}), 
\mathbf{f}_\mathbf{\Sigma} (\mathbf{x};\tilde{\boldsymbol{\lambda}})) \\
\mathbf{z}^{(i)} &= \mathbf{f}_{\boldsymbol{\mu}} (\mathbf{x};\tilde{\boldsymbol{\lambda}})
+ \mathbf{f}_{\boldsymbol{\Sigma}} (\mathbf{x};\tilde{\boldsymbol{\lambda}})^{1/2} \boldsymbol{\epsilon}^{(i)}
\;\;\;,\;\;\; \boldsymbol{\epsilon}^{(i)} \sim \mathcal{N}(\mathbf{0},\mathbf{I})
\end{align}

>* **As a result,**

>$$\mathcal{L}(\boldsymbol{\lambda}) \geq
\langle \log (p(\mathbf{x}|
\mathbf{f}_{\boldsymbol{\mu}} (\mathbf{x};\tilde{\boldsymbol{\lambda}})
+ \mathbf{f}_{\boldsymbol{\Sigma}} (\mathbf{x};\tilde{\boldsymbol{\lambda}})^{1/2} \boldsymbol{\epsilon}
;\boldsymbol{\lambda})) \rangle_{\mathcal{N}(\mathbf{0},\mathbf{I})}
+ \Bigg\langle \log \left( \frac{p(\mathbf{z})}
{q(\mathbf{z}|\mathbf{x};\tilde{\boldsymbol{\lambda}})} 
\right) \Bigg\rangle_{q(\mathbf{z}|\mathbf{x};\tilde{\boldsymbol{\lambda}})}
$$

# 2. Ensemble Methods

## 2.1. Introduction

* **Majority Voting**

>$$P(\text{error}) = \sum^N_{i=\frac{N}{2}} \left( \begin{matrix} N \\ i \end{matrix} \right)
p^i_e (1-p_e)^{N-i}$$

* **Bayesian Approaches** and **Monte-Carlo Approximation**

>* Consider an ensemble of **discriminative classifiers**

>\begin{align}
\hat{\omega} &= \text{argmax}_{{\omega}} \left\{ 
\sum_{\mathcal{M}} \int P({\omega}|\mathbf{x}^\star;\boldsymbol{\theta},\mathcal{M})
p(\boldsymbol{\theta}|\mathcal{M},\mathcal{D})
P(\mathcal{M}|\mathcal{D})d\boldsymbol{\theta} \right\} \\
&\approx \text{argmax}_{{\omega}} \left\{ 
\frac{1}{N} \sum^N_{j=1} P({\omega}|\mathbf{x}^\star;\mathcal{M}^{(j)})
\right\} \;\;\;,\;\;\;
\mathcal{M}^{(i)} \sim p(\boldsymbol{\theta},\mathcal{M}|\mathcal{D})
\end{align}

## 2.2. Ensemble Generation

* **Random Network Initialization** (each $\mathcal{M}{(i)}$ at local optimum)

>\begin{align}
\tilde{\mathcal{M}}^{(i)} &\sim p(\boldsymbol{\theta}|\mathcal{M}) 
\;\;\;\leftarrow\;\;\; \text{prior over parameters}\\
\mathcal{M}^{(i)} &= \text{argmax}_{\boldsymbol{\theta}} \left\{ 
\sum^n_{j=1} \log \left( P \left( y_j | \mathbf{x}_j ; \boldsymbol{\theta}, \tilde{\mathcal{M}}^{(i)} \right)
\right)
\right\}
\end{align}

* **Bagging** - train each model on a random subset of data $\tilde{\mathcal{D}}$
* **Monte Carlo Dropout** - de-activate random nodes
* **Adhoc Models** - number/size of layers, activation fn., cost function, ...

## 2.3. Model Compression: Teacher-Student Training

* **Cross Entropy Training Criterion** 

>$$\mathcal{D}=\{\mathbf{x}_{1:n}, y_{1:n}\} \;\;\;,\;\;\; y_i \in \{\omega_1,...,\omega_K\}$$

>$$\mathcal{F}_{ce} = - \sum^n_{i=1} \log P(y_i|\mathbf{x}_i;\mathcal{M}_S)
= - \sum^n_{i=1} \sum_{\omega} \delta(y_i,\omega) \log P(\omega|\mathbf{x}_i;\mathcal{M}_S)$$

>* $\delta(y_i,\omega)$: Kronecker delta function, sum over all classes

* **Modify Targets based on a Teacher Network** (soft targets)

>$$\mathcal{F}_{ts} = - \sum^n_{i=1} \sum_{\omega} 
P(\omega|\mathbf{x}_i;\mathcal{M}_T)
\log P(\omega|\mathbf{x}_i;\mathcal{M}_S)$$

* **Replace the Teacher by an Ensemble**

>$$\mathcal{F}_{ts} = - \sum^n_{i=1} \sum_{\omega} 
\left[
\frac{1}{N} \sum^N_{j=1} P \left( \omega|\mathbf{x}_i;\mathcal{M}^{(j)} \right)
\right]
\log P(\omega|\mathbf{x}_i;\mathcal{M}_S)$$

# 3. Kernels for Structured Data

>1. **Extract features from input objects**
1. **Compute kernels (dot products) using those features**

## 3.1. String Kernels

* **String Kernel**

>1. Given a list of substrings $s_1,s_2,... \in \mathcal{A}^\star$, encode $\boldsymbol{\phi}(x) = (\phi_{s_1}(x), \phi_{s_2}(x), ...)^T$
>1. $\phi_{s}(x)$: indicate the occurrence of $s$ in $x$
>1. Kernel between two strings defined as dot product:

>$$k(x,x') = \boldsymbol{\phi}(x)^T \boldsymbol{\phi}(x') 
= \sum_{s\in\{s_1,s_2,...\}} \phi_s(x) \phi_s(x')$$

* **Example: Gap-weighted Kernel**

>* $\phi_{s}(x)$: # occurrences, $\lambda^n$ for $n$ gaps

* **Example: $k$-spectrum kernel**

>* $\phi_{s}(x)$: # **exact** occurrences ($s$: any possible length-$k$ sequence)
>* Selection of $k$
>  * Large $k$: co-occurrence is more informative
>  * Small $k$: # co-occurrence increases
>  * $k=1$: **Bag-of-words kernel**

## 3.2. Tree Kernels

* **Definitions**

>* **Subtree:** tree formed by selecting one node an all its descendants
>* **Subset Tree:** subtree including all children of a node or none of them

* **Tree Kernel**

>1. Given a list of all possible subset trees $t_1,t_2,...$, each tree $\mathcal{T}$ is encoded using the feature vector $\boldsymbol{\phi}(\mathcal{T}) = (\phi_{t_1}(\mathcal{T}), \phi_{t_2}(\mathcal{T}), ...)^T$
>2. Each $\phi_{t}(\mathcal{T})$ counts occurrences of $t$ in $\mathcal{T}$ ($\mathcal{V}(\mathcal{T})$: set of nodes in $\mathcal{T}$)
>$$$$
>$$\phi_t (\mathcal{T}) = \sum_{n \in \mathcal{V}(\mathcal{T})} I_t (n)
\;\;\;,\;\;\;
I_t(n) = \bigg\{ \begin{matrix} 1 & t \text{ found in } \mathcal{T} \text{ with root at node } n \\ 0 & \text{otherwise} \end{matrix} $$
>$$$$
>3. Kernel between two trees defined as dot product:

>$$k(\mathcal{T}_a,\mathcal{T}_b) = \boldsymbol{\phi}(\mathcal{T}_a)^T \boldsymbol{\phi}(\mathcal{T}_b) = \sum_{t \in \{t_1,t_2,...\}} \phi_t(\mathcal{T}_a) \phi_t(\mathcal{T}_b)$$

* **Efficient Computation of Tree Kernels and Example**

>\begin{align}
k(\mathcal{T}_a,\mathcal{T}_b) &= \sum_{n_a \in \mathcal{V}(\mathcal{T}_a)}
\sum_{n_b \in \mathcal{V}(\mathcal{T}_b)} f(n_a, n_b) \\
\\
f(n_a,n_b) &= \sum_{t \in \{t_1,t_2,...\}} I_t(n_a) I_t(n_b)
\end{align}

>* $f(n_a,n_b)$: # common subset trees at $n_a$ and $n_b$

>|$\hspace{40mm}$Condition|                         Then|
|-|-|
|$n_a \neq n_b$ **or** $\text{ch}(n_a) \neq \text{ch}(n_b)$|$f(n_a,n_b)=0$|
|else if $n_a$ and $n_b$ are leaf nodes|$f(n_a,n_b)=1$|
|otherwise|$f(n_a,n_b)=\prod^{|\text{ch}(n_a)|}_{i=1} g(\text{ch}(n_a)_i,\text{ch}(n_b)_i)$<br/><br/>$g(n_1,n_2) = \bigg\{ \begin{matrix} 1 & \text{if } n_1 \text{ or } n_2 \text{ leaf} \\ 1+f(n_1,n_2) & \text{otherwise} \end{matrix}$|

## 3.3. Graph Kernels

* **Graphs and Graph Walks**

>* **Graph:** $\mathcal{G}=\{ \mathcal{V},\mathcal{E} \}$ where $\mathcal{E} = \{ (i,j);i,j\in\mathcal{V} \}$
>* **Graph Walks:** $k$-length walk defined as $w=\{v_1,...,v_{k+1}\}$ where $(v_i,v_{i+1}) \in \mathcal{E}$
>* Number of $k$-length walks between nodes $i$ and $j$:

>$$[\mathbf{A}^k]_{i,j} = \sum^{|\mathcal{V}|}_{s_1 = 1} \cdots \sum^{|\mathcal{V}|}_{s_{k-1} = 1} a_{i,s_1} a_{s_1,s_2} \cdot a_{s_{k-1},j}$$

* **Random-walk Graph Kernel**

>* $k(\mathcal{G},\mathcal{G}')$: # common walks in the two graphs
>* Computed using **direct product graph**

>\begin{align}
\mathcal{G}_\times &= (\mathcal{V}_\times, \mathcal{E}_\times) \\
\mathcal{V}_\times &= \{ (a,a') ; a\in\mathcal{V} \text{ and } a'\in\mathcal{V}' \} \\
\mathcal{E}_\times &= \{ ((a,a'),(b,b')) ; (a,b)\in\mathcal{E} \text{ and } (a',b')\in\mathcal{E}' \}
\end{align}

>* **Kernel Definition** ($\mathbf{A}_\times = \mathbf{A} \otimes \mathbf{A}'$ where $\otimes$ is the **Kronecker product**)

>$$k(\mathcal{G},\mathcal{G}') = \sum^{|\mathcal{V}_\times|}_{i,j = 1}
\left[ \sum^\infty_{n=0} \lambda^n \mathbf{A}^n_\times \right]_{i,j}
= \mathbf{1}^T [\mathbf{I} - \lambda \mathbf{A} \otimes \mathbf{A}']^{-1} \mathbf{1}
\Leftrightarrow \mathbf{x} = \mathbf{1} + \lambda (\mathbf{A} \otimes \mathbf{A}')\mathbf{x}$$

>* **Problems:** 
>  * A walk can visit the same cycle multiple times $\rightarrow$ small structural similarities can produce huge kernel values
>  * High cost, $\mathcal{O}(n^3)$

* **Weisfeiler-Lehman Graph Kernel**

>1. Create a set with labels of adjacent vertices & sort
>1. Add vertex label as a prefix
>1. Compress resulting label sequence into a **unique value**
>1. Assign the unique value as new **vertex label**
>1. Apply the **bag-of-words** kernel to the vertex labels (include the initial labels)

## 3.4. Fisher Kernel

* **Idea**

>* Use a **probabilistic generative model** to obtain a **fixed-length vector representation** of complex structured data

* **Steps**

>1. Train $p(\mathbf{x}|\boldsymbol{\theta})$ on $\{\mathbf{x}_n\}^N_{n=1}$ (e.g. using MLE)
>1. Define the **Fisher score vector** as $\boldsymbol{\phi}(\mathbf{x}_n) = \nabla_{\boldsymbol{\theta}} \log p(\mathbf{x}_n|\boldsymbol{\theta}) |_{\boldsymbol{\theta}_{\text{MLE}}}$
>1. Define **naive Fisher kernel** as $k(\mathbf{x}_n,\mathbf{x}_m) = \boldsymbol{\phi}(\mathbf{x}_n)^T \boldsymbol{\phi}(\mathbf{x}_m)$

* **Example: Mixture of Gaussians**

>\begin{align}
p(\mathbf{x}|\boldsymbol{\theta}) &= \sum^K_{k=1} p(\mathbf{x}|\theta_k)\pi_k \\
[\boldsymbol{\phi}(\mathbf{x}_n)]_{pi_k} &= \frac{\partial \log p(\mathbf{x}|\boldsymbol{\theta})}{\partial \pi_k} \bigg|_{\boldsymbol{\theta}=\boldsymbol{\theta}_{\text{MLE}}}
= \frac{p(\mathbf{x}|\theta^{\text{MLE}}_k)}{\sum_k p(\mathbf{x}|\theta^{\text{MLE}}_k) \pi^{\text{MLE}}_k}
\end{align}

>* $[\boldsymbol{\phi}(\mathbf{x}_n)]_{pi_k}$: amount by which the $k$-th component contributes to generate $\mathbf{x}_n$